In [67]:
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.stats as scp
import plotly.express as px
import matplotlib.pyplot as plt

In [68]:
df = pd.read_csv('../dataset final/final_dataset.csv')
df = df.drop(['Kills', 'Deaths', 'Assists', 'PlusMinus', 'ADR'], axis=1)
df

,GameID,PlayerName,TeamAbbreviation,Agent,ACS,HS_Percent,FirstKills,FirstDeaths,FKFD_PlusMinus,Num_2Ks,...,Patch,EventID,EventName,EventStage,Team1ID_y,Team2ID_y,Team1_y,Team2_y,Team1_MapScore,Team2_MapScore
0,60894,Reduxx,Boos,jett,313.0,31.0,4.0,4.0,0.0,2.0,...,3.0,826,Nerd Street Gamers Winter Championship - Regio...,Group Stage: Decider (A),6903,6020,Booster Seat Gaming,Pho Real,2,1
1,60894,ChurmZ,Boos,chamber,227.0,16.0,1.0,1.0,0.0,3.0,...,3.0,826,Nerd Street Gamers Winter Championship - Regio...,Group Stage: Decider (A),6903,6020,Booster Seat Gaming,Pho Real,2,1
2,60894,diaamond,Boos,sova,226.0,27.0,3.0,0.0,3.0,1.0,...,3.0,826,Nerd Street Gamers Winter Championship - Regio...,Group Stage: Decider (A),6903,6020,Booster Seat Gaming,Pho Real,2,1
3,60894,Boltzy,Boos,viper,218.0,19.0,3.0,0.0,3.0,3.0,...,3.0,826,Nerd Street Gamers Winter Championship - Regio...,Group Stage: Decider (A),6903,6020,Booster Seat Gaming,Pho Real,2,1
4,60894,Virtyy,Boos,skye,80.0,22.0,3.0,1.0,2.0,1.0,...,3.0,826,Nerd Street Gamers Winter Championship - Regio...,Group Stage: Decider (A),6903,6020,Booster Seat Gaming,Pho Real,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108245,2309,QUERNI,GRG,omen,227.0,18.0,1.0,0.0,1.0,3.0,...,1.0,68,Mandatory.gg Cup #2,Tournament: Round of 128,545,560,ISOL-BULIL,GRANIT GAMING,1,0
108246,2309,bmblbe,GRG,phoenix,110.0,33.0,0.0,2.0,-2.0,1.0,...,1.0,68,Mandatory.gg Cup #2,Tournament: Round of 128,545,560,ISOL-BULIL,GRANIT GAMING,1,0
108247,2309,DcTzR,GRG,jett,98.0,13.0,1.0,7.0,-6.0,0.0,...,1.0,68,Mandatory.gg Cup #2,Tournament: Round of 128,545,560,ISOL-BULIL,GRANIT GAMING,1,0
108248,2309,FRESHY,GRG,sova,91.0,19.0,0.0,1.0,-1.0,0.0,...,1.0,68,Mandatory.gg Cup #2,Tournament: Round of 128,545,560,ISOL-BULIL,GRANIT GAMING,1,0


## Penurunan Average ACS on Matches

In [69]:
df_1 = df[['GameID', 'TeamAbbreviation', 'ACS']]
df_1

,GameID,TeamAbbreviation,ACS
0,60894,Boos,313.0
1,60894,Boos,227.0
2,60894,Boos,226.0
3,60894,Boos,218.0
4,60894,Boos,80.0
...,...,...,...
108245,2309,GRG,227.0
108246,2309,GRG,110.0
108247,2309,GRG,98.0
108248,2309,GRG,91.0


In [70]:
df_team_1 = df_1[df_1.index % 10 < 5]
df_team_1 = df_team_1.groupby(by=['GameID', 'TeamAbbreviation']).mean().reset_index()
df_team_1 = df_team_1.rename(columns={'TeamAbbreviation': 'Team_1', 'ACS': 'ACS_1'})
df_team_1

,GameID,Team_1,ACS_1
0,2238,G2,227.4
1,2241,BBL,234.0
2,2244,forZ,204.4
3,2297,G2,231.0
4,2302,BBL,235.8
...,...,...,...
10820,60904,RAD,244.8
10821,60912,Salt,218.0
10822,60913,Salt,243.0
10823,60921,Mugi,203.6


In [71]:
df_team_2 = df_1[df_1.index % 10 >= 5]
df_team_2 = df_team_2.groupby(by=['GameID', 'TeamAbbreviation']).mean().reset_index()
df_team_2 = df_team_2.rename(columns={'TeamAbbreviation': 'Team_2', 'ACS': 'ACS_2'})
df_team_2

,GameID,Team_2,ACS_2
0,2238,ex-G,199.4
1,2241,RixGG,186.4
2,2244,SFCS,181.6
3,2297,G.G,151.0
4,2302,nmDM,211.4
...,...,...,...
10833,60904,same,172.6
10834,60912,Mugi,207.8
10835,60913,Mugi,148.0
10836,60921,CLG,165.0


In [72]:
df_games = df_team_1.merge(df_team_2, left_on='GameID', right_on='GameID')
df_games

,GameID,Team_1,ACS_1,Team_2,ACS_2
0,2238,G2,227.4,ex-G,199.4
1,2241,BBL,234.0,RixGG,186.4
2,2244,forZ,204.4,SFCS,181.6
3,2297,G2,231.0,G.G,151.0
4,2302,BBL,235.8,nmDM,211.4
...,...,...,...,...,...
10833,60904,RAD,244.8,same,172.6
10834,60912,Salt,218.0,Mugi,207.8
10835,60913,Salt,243.0,Mugi,148.0
10836,60921,Mugi,203.6,CLG,165.0


In [97]:
df_matches = df[['MatchID', 'GameID']].drop_duplicates().reset_index().drop('index', axis=1).merge(df_games, left_on='GameID', right_on='GameID')
df_matches = df_matches.drop(['GameID', 'Team_1', 'Team_2'], axis=1).groupby(['MatchID']).mean().reset_index()
df_matches

,MatchID,ACS_1,ACS_2
0,1448,227.400000,199.4
1,1451,234.000000,186.4
2,1454,204.400000,181.6
3,1490,231.000000,151.0
4,1496,235.800000,211.4
...,...,...,...
5189,62393,186.533333,200.2
5190,62394,238.300000,153.9
5191,62396,243.500000,150.9
5192,62399,230.500000,177.9
